In [1]:
# Application을 기준으로 대출했는지 안했는지만 체크

In [1]:
from glob import glob
from tqdm import tqdm
#from rdkit import Chem
import numpy as np
import pandas as pd
import re
import shutil
import os

import autogluon
from autogluon.common.utils.utils import setup_outputdir
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path
import sklearn
from sklearn.model_selection import train_test_split
import autogluon.core as ag
import pandas as pd
import numpy as np

import pandas as pd
import random
import argparse
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

import glob

/opt/conda/envs/bigcon/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
full_data = pd.read_csv("./dataset/full_data.csv")

In [3]:
# le_income_type

le_purpose_type = LabelEncoder()
le_purpose_type = le_purpose_type.fit(full_data["purpose"])
le_purpose_fitted = le_purpose_type.fit_transform(full_data["purpose"])
purpose_inverted = le_purpose_type.inverse_transform(le_purpose_fitted)

print(le_purpose_type.classes_)

['기타' '대환대출' '사업자금' '생활비' '자동차구입' '전월세보증금' '주택구입' '투자']


In [4]:
full_data["purpose"] = le_purpose_fitted

In [5]:
full_data["bank_id"] = full_data["bank_id"].astype('category')
full_data["product_id"] = full_data["product_id"].astype('category')
full_data["loan_limit"] = full_data["loan_limit"].astype('category')
full_data["birth_year"] = full_data["birth_year"].astype('category')

full_data["gender"] = full_data["gender"].astype('category')
full_data["credit_score"] = full_data["credit_score"].astype('category')
full_data["yearly_income"] = full_data["yearly_income"].astype('category')
full_data["income_type"] = full_data["income_type"].astype('category')
full_data["company_enter_month"] = full_data["company_enter_month"].astype('category')

full_data["employment_type"] = full_data["employment_type"].astype('category')
full_data["houseown_type"] = full_data["houseown_type"].astype('category')
full_data["desired_amount"] = full_data["desired_amount"].astype('category')
full_data["existing_loan_cnt"] = full_data["existing_loan_cnt"].astype('category')
full_data["existing_loan_amt"] = full_data["existing_loan_amt"].astype('category')

full_data["credit_score_nan"] = full_data["credit_score_nan"].astype('category')
full_data["gender_birth_nan"] = full_data["gender_birth_nan"].astype('category')
full_data["loan_limit/max_loan_limit_binary"] = full_data["loan_limit/max_loan_limit_binary"].astype('category')
full_data["loan_rate/min_loan_rate_binary"] = full_data["loan_rate/min_loan_rate_binary"].astype('category')

full_data["desired_amount/existing_loan_amt_binary"] = full_data["desired_amount/existing_loan_amt_binary"].astype('category')
full_data["hour"] = full_data["hour"].astype('category')
full_data["dayofweek"] = full_data["dayofweek"].astype('category')
full_data["purpose"] = full_data["purpose"].astype('category')
full_data["existing_loan_cnt_binary"] = full_data["existing_loan_cnt_binary"].astype("category")

In [6]:
full_data = full_data.drop(["application_id", "user_id"], axis = 1)

In [7]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4768898 entries, 0 to 4768897
Data columns (total 32 columns):
 #   Column                                   Dtype   
---  ------                                   -----   
 0   bank_id                                  category
 1   product_id                               category
 2   loan_limit                               category
 3   loan_rate                                float64 
 4   is_applied                               float64 
 5   birth_year                               category
 6   gender                                   category
 7   credit_score                             category
 8   yearly_income                            category
 9   income_type                              category
 10  company_enter_month                      category
 11  employment_type                          category
 12  houseown_type                            category
 13  desired_amount                           category
 14  pu

In [8]:
full_data.loc[full_data[full_data['desired_amount/existing_loan_amt'] >= 3].index, 'desired_amount/existing_loan_amt'] = 3

In [9]:
print("loan_rate")
print("q2",full_data['loan_rate'].median())
print("q3", full_data['loan_rate'].quantile(0.75))
print("q1", full_data['loan_rate'].quantile(0.25))
print("==")
print("loan_limit/max_loan_limit")
print("q2",full_data['loan_limit/max_loan_limit'].median())
print("q3", full_data['loan_limit/max_loan_limit'].quantile(0.75))
print("q1", full_data['loan_limit/max_loan_limit'].quantile(0.25))
print("==")
print("loan_rate/min_loan_rate")
print("q2",full_data['loan_rate/min_loan_rate'].median())
print("q3", full_data['loan_rate/min_loan_rate'].quantile(0.75))
print("q1", full_data['loan_rate/min_loan_rate'].quantile(0.25))
print("==")
print("desired_amount/existing_loan_amt")
print("q2",full_data['desired_amount/existing_loan_amt'].median())
print("q3", full_data['desired_amount/existing_loan_amt'].quantile(0.75))
print("q1", full_data['desired_amount/existing_loan_amt'].quantile(0.25))

loan_rate
q2 13.4
q3 15.9
q1 9.5
==
loan_limit/max_loan_limit
q2 0.4305555495756173
q3 0.6999999860000002
q1 0.2142857112244898
==
loan_rate/min_loan_rate
q2 1.8115942028985503
q3 2.4375
q1 1.2594936708860758
==
desired_amount/existing_loan_amt
q2 0.7476635537601537
q3 3.0
q1 0.2362204754479508


In [10]:
full_data['loan_rate'] = (full_data['loan_rate'] - full_data['loan_rate'].median() )/ (full_data['loan_rate'].quantile(0.75) - full_data['loan_rate'].quantile(0.25))
full_data['loan_limit/max_loan_limit'] = (full_data['loan_limit/max_loan_limit'] - full_data['loan_limit/max_loan_limit'].median() )/ (full_data['loan_limit/max_loan_limit'].quantile(0.75) - full_data['loan_limit/max_loan_limit'].quantile(0.25))
full_data['loan_rate/min_loan_rate'] = (full_data['loan_rate/min_loan_rate'] - full_data['loan_rate/min_loan_rate'].median() )/ (full_data['loan_rate/min_loan_rate'].quantile(0.75) - full_data['loan_rate/min_loan_rate'].quantile(0.25))

In [11]:
print("loan_rate")
print("q2",full_data['loan_rate'].median())
print("q3", full_data['loan_rate'].quantile(0.75))
print("q1", full_data['loan_rate'].quantile(0.25))
print("==")
print("loan_limit/max_loan_limit")
print("q2",full_data['loan_limit/max_loan_limit'].median())
print("q3", full_data['loan_limit/max_loan_limit'].quantile(0.75))
print("q1", full_data['loan_limit/max_loan_limit'].quantile(0.25))
print("==")
print("loan_rate/min_loan_rate")
print("q2",full_data['loan_rate/min_loan_rate'].median())
print("q3", full_data['loan_rate/min_loan_rate'].quantile(0.75))
print("q1", full_data['loan_rate/min_loan_rate'].quantile(0.25))
print("==")
print("desired_amount/existing_loan_amt")
print("q2",full_data['desired_amount/existing_loan_amt'].median())
print("q3", full_data['desired_amount/existing_loan_amt'].quantile(0.75))
print("q1", full_data['desired_amount/existing_loan_amt'].quantile(0.25))

loan_rate
q2 0.0
q3 0.390625
q1 -0.609375
==
loan_limit/max_loan_limit
q2 0.0
q3 0.5547385580728834
q1 -0.44526144192711664
==
loan_rate/min_loan_rate
q2 0.0
q3 0.531326344886657
q1 -0.468673655113343
==
desired_amount/existing_loan_amt
q2 0.7476635537601537
q3 3.0
q1 0.2362204754479508


In [12]:
full_data.columns

Index(['bank_id', 'product_id', 'loan_limit', 'loan_rate', 'is_applied',
       'birth_year', 'gender', 'credit_score', 'yearly_income', 'income_type',
       'company_enter_month', 'employment_type', 'houseown_type',
       'desired_amount', 'purpose', 'existing_loan_cnt', 'existing_loan_amt',
       'num_not_approve', 'num_total_approve', 'num_approve',
       'num_total_login', 'loan_limit/max_loan_limit',
       'loan_rate/min_loan_rate', 'desired_amount/existing_loan_amt',
       'credit_score_nan', 'gender_birth_nan', 'existing_loan_cnt_binary',
       'loan_limit/max_loan_limit_binary', 'loan_rate/min_loan_rate_binary',
       'desired_amount/existing_loan_amt_binary', 'hour', 'dayofweek'],
      dtype='object')

In [13]:
print("bank_id :" , len(full_data["bank_id"].unique()) + 1)
print("product_id : " , len(full_data["product_id"].unique()) + 1)
print("loan_limit : " , len(full_data["loan_limit"].unique()) + 1)
print("birth_year : " , len(full_data["birth_year"].unique()) + 1)
print("gender : " , len(full_data["gender"].unique()) + 1)
print("credit_score : " , len(full_data["credit_score"].unique()) + 1)
print("yearly_income : " , len(full_data["yearly_income"].unique()) + 1)
print("income_type : " , len(full_data["income_type"].unique()) + 1)
print("company_enter_month : " , len(full_data["company_enter_month"].unique()) + 1)
print("employment_type : " , len(full_data["employment_type"].unique()) + 1)
print("houseown_type : " , len(full_data["houseown_type"].unique()) + 1)
print("desired_amount : " , len(full_data["desired_amount"].unique()) + 1)
print("purpose : " , len(full_data["purpose"].unique()) + 1)
print("existing_loan_cnt : " , len(full_data["existing_loan_cnt"].unique()) + 1)
print("existing_loan_amt : " , len(full_data["existing_loan_amt"].unique()) + 1)
print("credit_score_nan : " , len(full_data["credit_score_nan"].unique()) + 1)
print("gender_birth_nan : " , len(full_data["gender_birth_nan"].unique()) + 1)
print("existing_loan_cnt_binary : " , len(full_data["existing_loan_cnt_binary"].unique()) + 1)
print("loan_limit/max_loan_limit_binary : " , len(full_data["loan_limit/max_loan_limit_binary"].unique()) + 1)
print("loan_rate/min_loan_rate_binary : " , len(full_data["loan_rate/min_loan_rate_binary"].unique()) + 1)
print("desired_amount/existing_loan_amt_binary : " , len(full_data["desired_amount/existing_loan_amt_binary"].unique()) + 1)
print("hour : " , len(full_data["hour"].unique()) + 1)
print("dayofweek : " , len(full_data["dayofweek"].unique()) + 1)
#print("existing_loan_cnt_binary: ", len(full_data["existing_loan_cnt_binary"].unique()) + 1)

bank_id : 63
product_id :  179
loan_limit :  52
birth_year :  15
gender :  4
credit_score :  25
yearly_income :  20
income_type :  7
company_enter_month :  10
employment_type :  5
houseown_type :  5
desired_amount :  20
purpose :  9
existing_loan_cnt :  11
existing_loan_amt :  20
credit_score_nan :  3
gender_birth_nan :  3
existing_loan_cnt_binary :  3
loan_limit/max_loan_limit_binary :  3
loan_rate/min_loan_rate_binary :  3
desired_amount/existing_loan_amt_binary :  4
hour :  25
dayofweek :  8


In [14]:
full_data_cat = full_data[["bank_id", "product_id", "loan_limit", "birth_year", "gender", "credit_score", "yearly_income", "income_type", "company_enter_month",\
                  "employment_type", "houseown_type", "desired_amount", "purpose", "existing_loan_cnt", "existing_loan_amt" , "credit_score_nan", "gender_birth_nan",\
                  "existing_loan_cnt_binary", "loan_limit/max_loan_limit_binary", "loan_rate/min_loan_rate_binary", "desired_amount/existing_loan_amt_binary", "hour", "dayofweek"]]

In [15]:
full_data_cont = full_data[['loan_rate', 'loan_limit/max_loan_limit', 'loan_rate/min_loan_rate', 'desired_amount/existing_loan_amt', 'num_not_approve', 'num_total_approve', 'num_approve', 'num_total_login']]

In [16]:
full_data_target = full_data[["is_applied"]]

In [17]:
full_data = pd.concat([full_data_cont, full_data_cat, full_data_target], axis = 1)

In [18]:
full_data

,loan_rate,loan_limit/max_loan_limit,loan_rate/min_loan_rate,desired_amount/existing_loan_amt,num_not_approve,num_total_approve,num_approve,num_total_login,bank_id,product_id,...,existing_loan_amt,credit_score_nan,gender_birth_nan,existing_loan_cnt_binary,loan_limit/max_loan_limit_binary,loan_rate/min_loan_rate_binary,desired_amount/existing_loan_amt_binary,hour,dayofweek,is_applied
0,0.484375,1.172386,-0.688956,0.061728,5,7,2,13,54,235,...,18,0,0,1,1,0,0,8,0,1.0
1,0.015625,-0.457516,0.310546,1.379310,5,7,2,13,42,216,...,12,0,0,0,0,0,1,10,0,0.0
2,0.390625,0.057190,0.639149,1.379310,0,2,2,8,25,169,...,12,0,0,0,1,1,1,10,0,0.0
3,0.781250,-0.457516,0.981444,1.379310,0,2,2,8,22,221,...,12,0,0,0,0,1,1,10,0,0.0
4,0.218750,-0.757761,0.488539,1.379310,0,2,2,8,44,8,...,12,0,0,0,0,0,1,10,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4768893,-1.000000,-0.236283,-0.623656,0.222222,0,1,1,0,27,148,...,6,0,0,0,1,0,0,6,0,0.0
4768894,-0.968750,-0.236283,-0.597537,0.222222,0,1,1,0,32,257,...,6,0,0,0,1,0,0,6,0,0.0
4768895,-0.546875,-0.308523,-0.244920,0.222222,0,1,1,0,8,31,...,6,0,0,0,0,0,0,6,0,0.0
4768896,0.265625,0.197153,0.434194,0.222222,0,1,1,0,49,39,...,6,0,0,0,1,0,0,6,0,0.0


In [19]:
def k_fold_result(data):
    ## Test set setting
    data = data.reset_index().iloc[:, 1:]
    

    #train, test = train_test_split(data, test_size=0.15,   random_state=20205289,  stratify=data['is_applied'])

    train = data.reset_index().iloc[:, 1:]
    #test = test.reset_index().iloc[:, 1:]

    from sklearn.model_selection import StratifiedKFold
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20205289)
    for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(train)), y=train['is_applied'])):
        train.loc[val_idx, 'fold'] = fold
        
    return train

In [20]:
train = k_fold_result(full_data)

In [21]:
fold_0 = train[train["fold"] == 0].iloc[:, :-1].reset_index().iloc[:, 1:]
fold_1 = train[train["fold"] == 1].iloc[:, :-1].reset_index().iloc[:, 1:]
fold_2 = train[train["fold"] == 2].iloc[:, :-1].reset_index().iloc[:, 1:]
fold_3 = train[train["fold"] == 3].iloc[:, :-1].reset_index().iloc[:, 1:]
fold_4 = train[train["fold"] == 4].iloc[:, :-1].reset_index().iloc[:, 1:]

In [22]:
fold_0.to_csv("./DL_dataset/fold_0.csv", index = False)
fold_1.to_csv("./DL_dataset/fold_1.csv", index = False)
fold_2.to_csv("./DL_dataset/fold_2.csv", index = False)
fold_3.to_csv("./DL_dataset/fold_3.csv", index = False)
fold_4.to_csv("./DL_dataset/fold_4.csv", index = False)

In [23]:
train.to_csv("./DL_dataset/train.csv", index = False)

In [24]:
train["fold"].unique()

array([4., 1., 2., 3., 0.])

In [25]:
train.iloc[:, :8]

,loan_rate,loan_limit/max_loan_limit,loan_rate/min_loan_rate,desired_amount/existing_loan_amt,num_not_approve,num_total_approve,num_approve,num_total_login
0,0.484375,1.172386,-0.688956,0.061728,5,7,2,13
1,0.015625,-0.457516,0.310546,1.379310,5,7,2,13
2,0.390625,0.057190,0.639149,1.379310,0,2,2,8
3,0.781250,-0.457516,0.981444,1.379310,0,2,2,8
4,0.218750,-0.757761,0.488539,1.379310,0,2,2,8
...,...,...,...,...,...,...,...,...
4768893,-1.000000,-0.236283,-0.623656,0.222222,0,1,1,0
4768894,-0.968750,-0.236283,-0.597537,0.222222,0,1,1,0
4768895,-0.546875,-0.308523,-0.244920,0.222222,0,1,1,0
4768896,0.265625,0.197153,0.434194,0.222222,0,1,1,0


In [26]:
train.iloc[:, 8:-2]

,bank_id,product_id,loan_limit,birth_year,gender,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,existing_loan_cnt,existing_loan_amt,credit_score_nan,gender_birth_nan,existing_loan_cnt_binary,loan_limit/max_loan_limit_binary,loan_rate/min_loan_rate_binary,desired_amount/existing_loan_amt_binary,hour,dayofweek
0,54,235,19,6,1,13,5,0,99,3,...,9,18,0,0,1,1,0,0,8,0
1,42,216,9,3,1,18,6,0,99,3,...,2,12,0,0,0,0,0,1,10,0
2,25,169,21,3,1,18,6,0,99,3,...,2,12,0,0,0,1,1,1,10,0
3,22,221,9,3,1,18,6,0,99,3,...,2,12,0,0,0,0,1,1,10,0
4,44,8,2,3,1,18,6,0,99,3,...,2,12,0,0,0,0,0,1,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4768893,27,148,17,5,0,14,5,1,11,1,...,3,6,0,0,0,1,0,0,6,0
4768894,32,257,17,5,0,14,5,1,11,1,...,3,6,0,0,0,1,0,0,6,0
4768895,8,31,15,5,0,14,5,1,11,1,...,3,6,0,0,0,0,0,0,6,0
4768896,49,39,29,5,0,14,5,1,11,1,...,3,6,0,0,0,1,0,0,6,0


In [27]:
train.shape

(4768898, 33)

In [31]:
l=[]
for i in range(8, 31):
    print(train.iloc[:, i:i+1].columns)
    l.append(len(train.iloc[:, i].astype(str).unique()) + 1)

Index(['bank_id'], dtype='object')
Index(['product_id'], dtype='object')
Index(['loan_limit'], dtype='object')
Index(['birth_year'], dtype='object')
Index(['gender'], dtype='object')
Index(['credit_score'], dtype='object')
Index(['yearly_income'], dtype='object')
Index(['income_type'], dtype='object')
Index(['company_enter_month'], dtype='object')
Index(['employment_type'], dtype='object')
Index(['houseown_type'], dtype='object')
Index(['desired_amount'], dtype='object')
Index(['purpose'], dtype='object')
Index(['existing_loan_cnt'], dtype='object')
Index(['existing_loan_amt'], dtype='object')
Index(['credit_score_nan'], dtype='object')
Index(['gender_birth_nan'], dtype='object')
Index(['existing_loan_cnt_binary'], dtype='object')
Index(['loan_limit/max_loan_limit_binary'], dtype='object')
Index(['loan_rate/min_loan_rate_binary'], dtype='object')
Index(['desired_amount/existing_loan_amt_binary'], dtype='object')
Index(['hour'], dtype='object')
Index(['dayofweek'], dtype='object')


In [32]:
l

[63,
 179,
 52,
 15,
 4,
 25,
 20,
 7,
 10,
 5,
 5,
 20,
 9,
 11,
 20,
 3,
 3,
 3,
 3,
 3,
 4,
 25,
 8]

In [34]:
l == [63, 179, 52, 15, 4, 25, 20, 7, 10, 5, 5, 20, 9, 11, 20, 3, 3, 3, 3, 3, 4, 25, 8]

True

In [35]:
train.columns

Index(['loan_rate', 'loan_limit/max_loan_limit', 'loan_rate/min_loan_rate',
       'desired_amount/existing_loan_amt', 'num_not_approve',
       'num_total_approve', 'num_approve', 'num_total_login', 'bank_id',
       'product_id', 'loan_limit', 'birth_year', 'gender', 'credit_score',
       'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'existing_loan_cnt', 'existing_loan_amt', 'credit_score_nan',
       'gender_birth_nan', 'existing_loan_cnt_binary',
       'loan_limit/max_loan_limit_binary', 'loan_rate/min_loan_rate_binary',
       'desired_amount/existing_loan_amt_binary', 'hour', 'dayofweek',
       'is_applied', 'fold'],
      dtype='object')

### Submit dataset

In [36]:
submit_test = pd.read_csv("./dataset/submit_test.csv")

In [37]:
# le_income_type

le_purpose_type = LabelEncoder()
le_purpose_type = le_purpose_type.fit(full_data["purpose"])
le_purpose_fitted = le_purpose_type.fit_transform(submit_test["purpose"])
purpose_inverted = le_purpose_type.inverse_transform(le_purpose_fitted)

print(le_purpose_type.classes_)

['기타' '대환대출' '사업자금' '생활비' '자동차구입' '전월세보증금' '주택구입' '투자']


In [39]:
submit_test["purpose"] = le_purpose_fitted

In [40]:
submit_test["bank_id"] = submit_test["bank_id"].astype('category')
submit_test["product_id"] = submit_test["product_id"].astype('category')
submit_test["loan_limit"] = submit_test["loan_limit"].astype('category')
submit_test["birth_year"] = submit_test["birth_year"].astype('category')
submit_test["gender"] = submit_test["gender"].astype('category')
submit_test["credit_score"] = submit_test["credit_score"].astype('category')
submit_test["yearly_income"] = submit_test["yearly_income"].astype('category')
submit_test["income_type"] = submit_test["income_type"].astype('category')
submit_test["company_enter_month"] = submit_test["company_enter_month"].astype('category')
submit_test["employment_type"] = submit_test["employment_type"].astype('category')
submit_test["houseown_type"] = submit_test["houseown_type"].astype('category')
submit_test["desired_amount"] = submit_test["desired_amount"].astype('category')
submit_test["existing_loan_cnt"] = submit_test["existing_loan_cnt"].astype('category')
submit_test["existing_loan_amt"] = submit_test["existing_loan_amt"].astype('category')
submit_test["credit_score_nan"] = submit_test["credit_score_nan"].astype('category')
submit_test["gender_birth_nan"] = submit_test["gender_birth_nan"].astype('category')
submit_test["loan_limit/max_loan_limit_binary"] = submit_test["loan_limit/max_loan_limit_binary"].astype('category')
submit_test["loan_rate/min_loan_rate_binary"] = submit_test["loan_rate/min_loan_rate_binary"].astype('category')
submit_test["desired_amount/existing_loan_amt_binary"] = submit_test["desired_amount/existing_loan_amt_binary"].astype('category')
submit_test["hour"] = submit_test["hour"].astype('category')
submit_test["dayofweek"] = submit_test["dayofweek"].astype('category')
submit_test["purpose"] = submit_test["purpose"].astype('category')
submit_test["existing_loan_cnt_binary"] = submit_test["existing_loan_cnt_binary"].astype("category")

In [41]:
submit_test = submit_test.drop(["application_id", "user_id"], axis = 1)

In [42]:
submit_test.loc[submit_test[submit_test['desired_amount/existing_loan_amt'] >= 3].index, 'desired_amount/existing_loan_amt'] = 3

In [43]:
submit_test['loan_rate'] = (submit_test['loan_rate'] - full_data['loan_rate'].median() )/ (full_data['loan_rate'].quantile(0.75) - full_data['loan_rate'].quantile(0.25))
submit_test['loan_limit/max_loan_limit'] = (submit_test['loan_limit/max_loan_limit'] - full_data['loan_limit/max_loan_limit'].median() )/ (full_data['loan_limit/max_loan_limit'].quantile(0.75) - full_data['loan_limit/max_loan_limit'].quantile(0.25))
submit_test['loan_rate/min_loan_rate'] = (submit_test['loan_rate/min_loan_rate'] - full_data['loan_rate/min_loan_rate'].median() )/ (full_data['loan_rate/min_loan_rate'].quantile(0.75) - full_data['loan_rate/min_loan_rate'].quantile(0.25))

In [44]:
submit_test_cat = submit_test[["bank_id", "product_id", "loan_limit", "birth_year", "gender", "credit_score", "yearly_income", "income_type", "company_enter_month",\
                  "employment_type", "houseown_type", "desired_amount", "purpose", "existing_loan_cnt", "existing_loan_amt" , "credit_score_nan", "gender_birth_nan",\
                  "existing_loan_cnt_binary", "loan_limit/max_loan_limit_binary", "loan_rate/min_loan_rate_binary", "desired_amount/existing_loan_amt_binary", "hour", "dayofweek"]]

In [45]:
submit_test_cont = submit_test[['loan_rate', 'loan_limit/max_loan_limit', 'loan_rate/min_loan_rate', 'desired_amount/existing_loan_amt', 'num_not_approve', 'num_total_approve', 'num_approve', 'num_total_login']]

In [46]:
submit_test_cont_target = submit_test[["is_applied"]]

In [47]:
submit_test = pd.concat([submit_test_cont, submit_test_cat, submit_test_cont_target], axis = 1)

In [48]:
submit_test["is_applied"] = 0

In [49]:
submit_test.columns

Index(['loan_rate', 'loan_limit/max_loan_limit', 'loan_rate/min_loan_rate',
       'desired_amount/existing_loan_amt', 'num_not_approve',
       'num_total_approve', 'num_approve', 'num_total_login', 'bank_id',
       'product_id', 'loan_limit', 'birth_year', 'gender', 'credit_score',
       'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'existing_loan_cnt', 'existing_loan_amt', 'credit_score_nan',
       'gender_birth_nan', 'existing_loan_cnt_binary',
       'loan_limit/max_loan_limit_binary', 'loan_rate/min_loan_rate_binary',
       'desired_amount/existing_loan_amt_binary', 'hour', 'dayofweek',
       'is_applied'],
      dtype='object')

In [50]:
train.columns

Index(['loan_rate', 'loan_limit/max_loan_limit', 'loan_rate/min_loan_rate',
       'desired_amount/existing_loan_amt', 'num_not_approve',
       'num_total_approve', 'num_approve', 'num_total_login', 'bank_id',
       'product_id', 'loan_limit', 'birth_year', 'gender', 'credit_score',
       'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'existing_loan_cnt', 'existing_loan_amt', 'credit_score_nan',
       'gender_birth_nan', 'existing_loan_cnt_binary',
       'loan_limit/max_loan_limit_binary', 'loan_rate/min_loan_rate_binary',
       'desired_amount/existing_loan_amt_binary', 'hour', 'dayofweek',
       'is_applied', 'fold'],
      dtype='object')

In [51]:
submit_test.to_csv("./DL_dataset/test.csv", index = False)

In [53]:
submit_test["purpose"]

0          1
1          1
2          1
3          1
4          1
          ..
3257234    1
3257235    1
3257236    1
3257237    1
3257238    1
Name: purpose, Length: 3257239, dtype: category
Categories (8, int64): [0, 1, 2, 3, 4, 5, 6, 7]